# Composability

This notebook shows off some basic functionality around LangChain Expression Language, which makes it really easy to compose arbitrary chains.

For a much deeper dive, see:

- [Full LangChain Expression Language Documentation](https://python.langchain.com/docs/expression_language/)

## Basic Composability

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the "light" of the party!'

### Batching

In [6]:
chain.batch([{"topic": "bear"}, {"topic": "clowns"}])

["Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!",
 'Why did the clown go to the doctor?\n\nBecause he was feeling a little funny!']

### Streaming

In [7]:
for s in chain.stream({"topic": "bears"}):
    print(s)


Why
 did
 the
 bear
 bring
 a
 flashlight
 to
 the
 party
?
 

Because
 he
 heard
 it
 was
 a
 "
bear
"
 necessity
!



## RunnablePassthrough

In [10]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

In [11]:
chain = prompt | model | output_parser

In [12]:
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform built by the langchain team.'

In [13]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

In [14]:
retrieval_chain.invoke({"question": "what is langsmith"})

'LangSmith is a platform that helps you build, debug, and deploy language model applications and intelligent agents. It provides features such as visualizing inputs and outputs, running experiments, and sharing results with the community.'

## RunnableParallel

In [15]:
from langchain_core.runnables import RunnableParallel

In [16]:
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser

In [17]:
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})

In [18]:
parallel_chain.invoke({"question": "what is langsmith"})

{'retrieved_answer': 'LangSmith is a platform that helps users debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It seamlessly integrates with LangChain, an open-source framework for building with LLMs. LangSmith allows users to trace and evaluate their language model applications and intelligent agents, providing interactive walkthroughs, documentation, cookbook, and feedback options to help move from prototype to production.',
 'simple_answer': 'Langsmith is a platform for language learners and teachers to connect and exchange knowledge. It offers resources, lessons, and practice exercises for various languages, as well as opportunities to connect with native speakers for language exchange.'}

In [19]:
for s in parallel_chain.stream({"question": "what is langsmith"}):
    print(s)

{'simple_answer': ''}
{'simple_answer': 'Lang'}
{'simple_answer': 'smith'}
{'simple_answer': ' is'}
{'simple_answer': ' a'}
{'simple_answer': ' surname'}
{'simple_answer': ' of'}
{'simple_answer': ' English'}
{'simple_answer': ' origin'}
{'simple_answer': '.'}
{'simple_answer': ' It'}
{'simple_answer': ' is'}
{'simple_answer': ' not'}
{'simple_answer': ' a'}
{'simple_answer': ' widely'}
{'simple_answer': ' known'}
{'simple_answer': ' or'}
{'simple_answer': ' common'}
{'simple_answer': ' surname'}
{'simple_answer': '.'}
{'simple_answer': ''}
{'retrieved_answer': ''}
{'retrieved_answer': 'Lang'}
{'retrieved_answer': 'Smith'}
{'retrieved_answer': ' is'}
{'retrieved_answer': ' a'}
{'retrieved_answer': ' platform'}
{'retrieved_answer': ' that'}
{'retrieved_answer': ' helps'}
{'retrieved_answer': ' you'}
{'retrieved_answer': ' debug'}
{'retrieved_answer': ','}
{'retrieved_answer': ' test'}
{'retrieved_answer': ','}
{'retrieved_answer': ' evaluate'}
{'retrieved_answer': ','}
{'retrieved_answe

In [17]:
result = {}
for s in parallel_chain.stream({"question": "what is langsmith"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

{'simple_answer': ''}
{'simple_answer': 'I'}
{'simple_answer': "I'm"}
{'simple_answer': "I'm sorry"}
{'simple_answer': "I'm sorry,"}
{'simple_answer': "I'm sorry, but"}
{'simple_answer': "I'm sorry, but I"}
{'simple_answer': "I'm sorry, but I couldn"}
{'simple_answer': "I'm sorry, but I couldn't"}
{'simple_answer': "I'm sorry, but I couldn't find"}
{'simple_answer': "I'm sorry, but I couldn't find any"}
{'simple_answer': "I'm sorry, but I couldn't find any information"}
{'simple_answer': "I'm sorry, but I couldn't find any information on"}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Lang'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith."'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "Langsmith." It'}
{'simple_answer': 'I\'m sorry, but I coul

{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility'}
{'simple_answer': 'I\'m sorry, but I couldn\'t find any information on "langsmith." It seems to be a term that is not widely recognized or used. Could you please provide more context or clarify your question?', 'retrieved_answer': 'LangSmith is a platform that offers features for debugging, testing, evaluating, and monitoring Language Learning Models (LLMs) and AI applications. It helps users trace and evaluate their language model applications and intelligent agents, providing visibility into'}
{'s